## --- Day 14: Extended Polymerization ---

Apply 10 steps of pair insertion to the polymer template and find the most and least common elements in the result. What do you get if you take the quantity of the most common element and subtract the quantity of the least common element?

In [1]:
from collections import Counter

class Polymerizer:
    def __init__(self, filename):
        self._rules = {}

        with open(filename) as f:
            # First line is template
            self._template = f.readline().strip()

            # Skip blank line, then read all pair insertion rules
            _ = f.readline()

            for line in f.readlines():
                elements, insert = line.strip().split(" -> ")
                self._rules[(elements[0], elements[1])] = insert

        self._polymer = self._template
        self._steps = 0

    def do_steps(self, num_steps, verbose=False):
        if verbose:
            print("Template:\t", self._template)
        for _ in range(num_steps):
            self.do_step()
            if verbose:
                print(f"After step {self._steps}:\t", self._polymer)

        return self.get_score()

    def do_step(self):
        self._steps += 1
        new_polymer = []

        for i in range(len(self._polymer)-1):
            new_polymer.append(self._polymer[i])
            if (self._polymer[i], self._polymer[i+1]) in self._rules:
                insert_element = self._rules[(self._polymer[i], self._polymer[i+1])]
                new_polymer.append(insert_element)
        new_polymer.append(self._polymer[-1])
        self._polymer = "".join(new_polymer)

    def get_score(self):
        counts = Counter(self._polymer).most_common()
        most, least = counts[0][1], counts[-1][1]
        
        return most - least



In [2]:
ex1_polymerizer = Polymerizer("./inputs/Day14ex.txt")
ex1_solution = 1588

assert ex1_solution == ex1_polymerizer.do_steps(10)

In [3]:
# Part 1 solution
p1_polymerizer = Polymerizer("./inputs/Day14.txt")
p1_polymerizer.do_steps(10)

2027

## --- Part Two ---
Apply 40 steps of pair insertion to the polymer template and find the most and least common elements in the result. What do you get if you take the quantity of the most common element and subtract the quantity of the least common element?

In [4]:
class BigPolymerizer:
    def __init__(self, filename):
        self._rules = {}
        self._pairs = {}
        self._steps = 0

        with open(filename) as f:
            # First line is template
            self._template = f.readline().strip()

            # Skip blank line, then read all pair insertion rules
            _ = f.readline()

            for line in f.readlines():
                elements, insert = line.strip().split(" -> ")
                self._rules[elements[0:2]] = insert

        for i in range(len(self._template)-1):
            pair = self._template[i:i+2]
            if pair in self._pairs:
                self._pairs[pair] += 1
            else:
                self._pairs[pair] = 1
    
    def do_step(self):
        new_pairs = []
        drop_pairs = []

        for pair, insert in self._rules.items():
            if pair in self._pairs:
                # Number of times this pair exists
                count = self._pairs[pair]

                # Split pair(s) no longer exist (after all rules applied)
                drop_pairs.append(pair)

                # (count) of 2 newly created pairs created (after rules applied)
                new_pairs.append((pair[0]+insert, count))
                new_pairs.append((insert+pair[1], count))

        # After all rules are processed, apply changes
        for pair in drop_pairs:
            del(self._pairs[pair])

        for new_pair in new_pairs:
            pair, count = new_pair
            if pair in self._pairs:
                self._pairs[pair] += count
            else:
                self._pairs[pair] = count

    def do_steps(self, num_steps):
        for _ in range(num_steps):
            self.do_step()

        return self.get_score()
    
    def count_elements(self):
        counts = {}
        for pair, count in self._pairs.items():
            if pair[0] in counts:
                counts[pair[0]] += count
            else:
                counts[pair[0]] = count
        
        # The last element of the input is never the left of a pair so add it manually
        if self._template[-1] in counts:
            counts[self._template[-1]] += 1
        else:
            counts[self._template[-1]] = 1

        
        return counts

    def get_score(self):
        el_counts = self.count_elements()
        most = max(el_counts.values())
        least = min(el_counts.values())

        return most - least

In [5]:
# Rerun example 1 with BigPolymerizer
ex1a_bigpoly = BigPolymerizer("./inputs/Day14ex.txt")
ex1a_solution = 1588

assert ex1a_solution == ex1a_bigpoly.do_steps(10)

In [6]:
# Rerun part 1 with BigPolymerizer
p1a_bigpoly = BigPolymerizer("./inputs/Day14.txt")
p1_solution = 2027

p1a_bigpoly.do_steps(10)

assert p1_solution == p1_solution

In [7]:
# Example 2
ex2_polymerizer = BigPolymerizer("./inputs/Day14ex.txt")
ex2_solution = 2188189693529

assert ex2_solution == ex2_polymerizer.do_steps(40)

In [8]:
# Part 2 solution
p2_polymerizer = BigPolymerizer("./inputs/Day14.txt")

p2_polymerizer.do_steps(40)

2265039461737